In [1]:
#read in all .json files
import os
DATA_LOC= f"..{os.sep}Data{os.sep}Pending"
file_ext=".json"
files=[]
for file in os.listdir(DATA_LOC):
    if file.endswith(file_ext):
        files.append(os.path.join(DATA_LOC, file))



In [2]:
#process a file
import json
facts=[]
misinformation=[]
temporal=[]
ignore=[]
opinion=[]
none=[]
errors=[]

for file in files:
    with open(file,encoding='utf8') as f:
        data = json.load(f)

        sentences=data['sentences']

        for sentence in sentences:
            annotated=False
            if sentence['fact'] and sentence['misinformation']:
                errors.append(sentence)
                annotated=True
            elif sentence['misinformation'] and sentence['opinion']:
                errors.append(sentence)
                annotated=True
            elif sentence['fact']:
                facts.append(sentence)
                annotated=True
            elif sentence['misinformation']:
                misinformation.append(sentence)
                annotated=True
            elif sentence['opinion']:
                opinion.append(sentence)
                annotated=True
            if sentence['temporal']:
                temporal.append(sentence)
                annotated=True
            if sentence['ignore']:
                ignore.append(sentence)
                annotated=True
            if not annotated:
                none.append(sentence)
           
    
print(f'# of Facts: {len(facts)}')
print(f'# of Misinformation: {len(misinformation)}')
print(f'# of Opinion: {len(opinion)}')
print(f'# of temporal: {len(temporal)}')
print(f'# of ignore: {len(ignore)}')
print(f'# of not annotated: {len(none)}')
print(f'# of errors: {len(errors)}')
    

# of Facts: 3221
# of Misinformation: 142
# of Opinion: 4108
# of temporal: 308
# of ignore: 3441
# of not annotated: 23173
# of errors: 64


In [3]:
from sklearn import svm
import random,math
from nltk.corpus import stopwords
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

stemmer = PorterStemmer()
def tokenize(text):
    text=text.lower()
    tokens = nltk.word_tokenize(text)
   # stems = []
   # for item in tokens:
   #     stems.append(PorterStemmer().stem(item))
    #return stems
    return tokens
def tokenize_stem(text):
    text=text.lower()
    tokens = nltk.word_tokenize(text)
    stems = []
    for item in tokens:
        stems.append(PorterStemmer().stem(item))
    return stems

def tokenize_pos(text):
    text=text.lower()
    tokens = nltk.word_tokenize(text)
    pos=nltk.pos_tag(tokens)
    tags=[]
    for p in pos:
        tags.append(p[1])
    return tags

def tokenize_pos_combo(text):
    text=text.lower()
    tokens = nltk.word_tokenize(text)
    pos=nltk.pos_tag(tokens)
    tags=[]
    for p in pos:
        tags.append(p[0])
        tags.append(p[1])
    return tags


data =[]
y=[]

#classification over 5 categories. Can make binary by only selecting two of the blocks below
data.extend([d['sentence'] for d in facts])
y.extend(['fact']*len(facts))

data.extend([d['sentence'] for d in opinion])
y.extend(['opinion']*len(opinion))

data.extend([d['sentence'] for d in misinformation])
y.extend(['misinformation']*len(misinformation))

data.extend([d['sentence'] for d in ignore])
y.extend(['ignore']*len(ignore))

data.extend([d['sentence'] for d in none])
y.extend(['none']*len(none))

labels = ['fact','opinion','misinformation','ignore','none']
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfconverter = TfidfVectorizer(max_features=1500, min_df=1, max_df=0.9,ngram_range=(1, 3),tokenizer=tokenize,)

X = tfidfconverter.fit_transform(data).toarray()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=100, random_state=0)
classifier.fit(X_train, y_train) 




RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [4]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
y_pred = classifier.predict(X_test)
a =  confusion_matrix(y_test, y_pred, labels=labels)

print(pd.DataFrame(a, index=labels, columns=labels))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))



                fact  opinion  misinformation  ignore  none
fact             233       62               0       7   329
opinion          104      141               0      21   557
misinformation     1        2               1       0    28
ignore            14       17               0     305   331
none             108       92               0      52  4412
                precision    recall  f1-score   support

          fact       0.51      0.37      0.43       631
        ignore       0.79      0.46      0.58       667
misinformation       1.00      0.03      0.06        32
          none       0.78      0.95      0.85      4664
       opinion       0.45      0.17      0.25       823

      accuracy                           0.75      6817
     macro avg       0.71      0.40      0.43      6817
  weighted avg       0.72      0.75      0.71      6817

0.7469561390641044
